In [1]:
import os
import math
from os import path

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19
from sklearn.model_selection import train_test_split
from PIL import Image

batch_size = 16

In [ ]:
tf.test.is_gpu_available()

# Data Loading

In [2]:
gitw_path = path.join("data", "GITW_clean")
classes = [d for d in os.listdir(gitw_path) if path.isdir(path.join(gitw_path, d))]
classes

['Background',
 'Bowl',
 'CanOfCocaCola',
 'FryingPan',
 'Glass',
 'Jam',
 'Lid',
 'MilkBottle',
 'Mug',
 'OilBottle',
 'Plate',
 'Rice',
 'SaucePan',
 'Sponge',
 'Sugar',
 'VinegarBottle',
 'WashLiquid']

In [3]:
directories = []
files = {
    "file": [],
    "label": [],
    "frame": []
}

for c_name in classes:
    for img in os.listdir(path.join(gitw_path, c_name)):
        if path.isfile(path.join(gitw_path, c_name, img)):
            files["file"].append(path.join(c_name, img))
            files["label"].append(classes.index(c_name))
            files["frame"].append(int(img.split("_")[1]))

In [4]:
files = pd.DataFrame(files).sort_values(by=["label", "frame"]).reset_index(drop=True)
files.head()

,file,label,frame
0,Background\Frame_0320_bg.png,0,320
1,Background\Frame_0360_bg.png,0,360
2,Background\Frame_0400_bg.png,0,400
3,Background\Frame_0440_bg.png,0,440
4,Background\Frame_0480_bg.png,0,480


In [5]:
x_train, x_test = train_test_split(files.index, test_size=0.15)
len(x_train), len(x_test)

(4451, 786)

In [6]:
train_df, test_df = files.loc[x_train], files.loc[x_test]
train_df.to_csv(path.join(gitw_path, "train_dataset.csv"), index=False)
test_df.to_csv(path.join(gitw_path, "test_dataset.csv"), index=False)

In [7]:
class GITWDataset(keras.utils.Sequence):
    
    def __init__(self, data_path, df, classes, batch_size=32):
        self.X = [path.join(data_path, file) for file in df["file"].values]
        self.Y = keras.utils.to_categorical(df["label"].values)
        self.batch_size = batch_size
        self.path = data_path
        self.df = df
        self.classes = classes
        
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
    
    def __getitem__(self, idx):
        files_x = self.X[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_y = self.Y[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_x = []
        for file in files_x:
            frame = np.array(Image.open(os.path.join(file)).convert("RGB").resize((224,224), Image.BICUBIC).getdata())
            batch_x.append(frame.reshape(224, 224, 3))
        return np.stack(batch_x), batch_y

In [8]:
trainset = GITWDataset(gitw_path, train_df, classes, batch_size=batch_size)
testset = GITWDataset(gitw_path, test_df, classes, batch_size=batch_size)

# ResNet fine-tuning

In [10]:
resnet_module = ResNet50(include_top=False, input_shape=(224, 224, 3), )
resnet = models.Sequential([
    resnet_module,
    layers.Flatten(),
    layers.Dense(len(classes), activation="softmax")
])

In [11]:
resnet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 17)                1706001   
Total params: 25,293,713
Trainable params: 25,240,593
Non-trainable params: 53,120
_________________________________________________________________


In [12]:
tensorboard = TensorBoard(log_dir=path.join(".", "logs", "resnet"), update_freq=50)
reducer = ReduceLROnPlateau(monitor='val_accuracy')

optimizer = optimizers.Adam(lr=1e-3, decay=1e-8)

resnet.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
resnet_history = resnet.fit_generator(trainset, 
                    epochs=20,
                    validation_data=testset,
                    callbacks=[tensorboard, reducer])

Epoch 1/20
279/279 [==============================] - 297s 1s/step - loss: 3.6505 - accuracy: 0.3876 - val_loss: 13.3320 - val_accuracy: 0.0980
Epoch 2/20
279/279 [==============================] - 292s 1s/step - loss: 2.2665 - accuracy: 0.6129 - val_loss: 2.4957 - val_accuracy: 0.5407
Epoch 3/20
279/279 [==============================] - 300s 1s/step - loss: 2.0806 - accuracy: 0.6457 - val_loss: 14.3799 - val_accuracy: 0.0980
Epoch 4/20
279/279 [==============================] - 289s 1s/step - loss: 2.1587 - accuracy: 0.6284 - val_loss: 1.5320 - val_accuracy: 0.6056
Epoch 5/20
279/279 [==============================] - 290s 1s/step - loss: 2.3707 - accuracy: 0.5904 - val_loss: 1.1454 - val_accuracy: 0.6858
Epoch 6/20
279/279 [==============================] - 293s 1s/step - loss: 1.8323 - accuracy: 0.7261 - val_loss: 3.4437 - val_accuracy: 0.6272
Epoch 7/20
279/279 [==============================] - 288s 1s/step - loss: 1.5407 - accuracy: 0.7965 - val_loss: 1.8656 - val_accuracy: 0.73

# VGG from random initialisation

In [10]:
vgg_module = VGG19(include_top=False, weights=None, input_shape=(224, 224, 3))
vgg = models.Sequential([
    vgg_module,
    layers.Flatten(),
    layers.Dense(len(classes), activation="softmax")
])

In [11]:
vgg.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 17)                426513    
Total params: 20,450,897
Trainable params: 20,450,897
Non-trainable params: 0
_________________________________________________________________


In [13]:
tensorboard = TensorBoard(log_dir=path.join(".", "logs", "vgg"), update_freq=50)
reducer = ReduceLROnPlateau(monitor='val_accuracy')

optimizer = optimizers.Adam(lr=1e-3, decay=1e-8)

vgg.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
vgg_history = vgg.fit_generator(trainset, 
                    epochs=20,
                    validation_data=testset,
                    callbacks=[tensorboard, reducer])

Epoch 1/20


ResourceExhaustedError: OOM when allocating tensor with shape[16,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2DBackpropInput]